<b><center> Homework 6 </center></b>
<br>
<b><center> James Bonifield </center></b>
<br>
<br>
<b>Question 9.1</b><br>
Using the same crime data set as in Question 8.2, apply Principal Component Analysis and then create a
regression model using the first few principal components. Specify your new model in terms of the
original variables (not the principal components), and compare its quality to that of your solution to
Question 8.2. You can use the R function prcomp for PCA. Note that to first scale the data, you can
include scale. = TRUE to scale as part of the PCA function. Don’t forget that, to make a prediction
for the new city, you’ll need to unscale the coefficients (i.e., do the scaling calculation in reverse!)

<b>First, we'll load the data, then perform PCA. We should probably exclude the binary data (column two), but I'm not really going to bother with this assignment </b>

In [171]:
library(tidyverse)
library(broom)
data <- suppressMessages((read_tsv('uscrime.txt')))

In [172]:
CrimeFactors <- data[,-16]
print(CrimeFactors)

pca.data <- prcomp(CrimeFactors ,scale.=TRUE, center=TRUE)

# A tibble: 47 x 15
       M    So    Ed   Po1   Po2    LF   M.F   Pop    NW     U1    U2 Wealth
   <dbl> <int> <dbl> <dbl> <dbl> <dbl> <dbl> <int> <dbl>  <dbl> <dbl>  <int>
 1  15.1     1  9.10  5.80  5.60 0.510  95.0    33 30.1  0.108   4.10   3940
 2  14.3     0 11.3  10.3   9.50 0.583 101      13 10.2  0.0960  3.60   5570
 3  14.2     1  8.90  4.50  4.40 0.533  96.9    18 21.9  0.0940  3.30   3180
 4  13.6     0 12.1  14.9  14.1  0.577  99.4   157  8.00 0.102   3.90   6730
 5  14.1     0 12.1  10.9  10.1  0.591  98.5    18  3.00 0.0910  2.00   5780
 6  12.1     0 11.0  11.8  11.5  0.547  96.4    25  4.40 0.0840  2.90   6890
 7  12.7     1 11.1   8.20  7.90 0.519  98.2     4 13.9  0.0970  3.80   6200
 8  13.1     1 10.9  11.5  10.9  0.542  96.9    50 17.9  0.0790  3.50   4720
 9  15.7     1  9.00  6.50  6.20 0.553  95.5    39 28.6  0.0810  2.80   4210
10  14.0     0 11.8   7.10  6.80 0.632 103       7  1.50 0.100   2.40   5260
# ... with 37 more rows, and 3 more variables: Ineq <dbl

In [173]:
summary(pca.data)
plot(pca.data)
plot(pca.data$sdev^2/sum(pca.data$sdev^2),
    xlab = "Principle Component",
    ylab = "Proportional Variance",
    ylim = c(0,1))

Importance of components%s:
                          PC1    PC2    PC3     PC4     PC5     PC6     PC7
Standard deviation     2.4534 1.6739 1.4160 1.07806 0.97893 0.74377 0.56729
Proportion of Variance 0.4013 0.1868 0.1337 0.07748 0.06389 0.03688 0.02145
Cumulative Proportion  0.4013 0.5880 0.7217 0.79920 0.86308 0.89996 0.92142
                           PC8     PC9    PC10    PC11    PC12    PC13   PC14
Standard deviation     0.55444 0.48493 0.44708 0.41915 0.35804 0.26333 0.2418
Proportion of Variance 0.02049 0.01568 0.01333 0.01171 0.00855 0.00462 0.0039
Cumulative Proportion  0.94191 0.95759 0.97091 0.98263 0.99117 0.99579 0.9997
                          PC15
Standard deviation     0.06793
Proportion of Variance 0.00031
Cumulative Proportion  1.00000

<b>Taking a look at the proportion of variance and the cumulative proportion, it seemed reasonable to use any components with a proportion of variance > 0.05, which yields the first 5 components that explain 86.3% of the total variance</b>

<b>Now, we perform regression on those PCs with the original Crime Data as the response:</b>

In [174]:
TrimmedCrime <- cbind(pca.data$x[,1:4],data=data[,16])
PCregModel <- lm(Crime~. ,data = as.data.frame(TrimmedCrime))
PCregModel$coefficients

(Intercept)         PC1         PC2         PC3         PC4 
  905.08511    65.21593   -70.08312    25.19408    69.44603

Now we transform the data back (unorthogonalize it? Not sure what the word for that is...) by using the following relationship $$X'=A'X$$
where $A'$ PC rotation matrix and $X$ is the matrix with the transformed regression coefficients

In [175]:
beta0 <- PCregModel$coefficients[1]
betas <- PCregModel$coefficients[2:5]

#Transform PCA data back to original
alphas <- pca.data$rotation[,1:4]
alphas <- alphas %*% betas

#Unscale Data 
UnscaledAlphas <- alphas/sapply(CrimeFactors,sd)
UnscaledBeta0 <- beta0 - sum(alphas*sapply(CrimeFactors,mean)/sapply(CrimeFactors,sd))



This allows us to see the original model in the terms of the original variables

In [176]:
print(UnscaledBeta0)
print(UnscaledAlphas)

(Intercept) 
   1666.485 
                [,1]
M        -16.9307630
So        21.3436771
Ed        12.8297238
Po1       21.3521593
Po2       23.0883154
LF      -346.5657125
M.F       -8.2930969
Pop        1.0462155
NW         1.5009941
U1     -1509.9345216
U2         1.6883674
Wealth     0.0400119
Ineq      -6.9020218
Prob     144.9492678
Time      -0.9330765


Those coefficients can be compared to the original Linear Regression Model derived in the previous weeks homework:

In [177]:
NewCity <- c(14.0,0.0,10.0,12.0,15.5,0.640,94.0,150.0,1.1,0.120,3.6,3200.0,20.1,0.04,39.0)

lm.Data <- tidy(lm(Crime~. , data=data))
print("Linear Coefficients")
print(lm.Data$estimate)

estimate <- c()

#Note we do i+1 on the lm.Data$estimate term because the first term is the intercept
for (i in 1:length(NewCity)){
    estimate[i] <- lm.Data$estimate[i+1] * NewCity[i]
}

EstimatedCrime <- sum(estimate) +lm.Data$estimate[1]
print("Estimated Crime for new City")
print(EstimatedCrime)
print("Estimated Crime for new City using new model")
print( NewCity %*% UnscaledAlphas + UnscaledBeta0)

[1] "Linear Coefficients"
 [1] -5.984288e+03  8.783017e+01 -3.803450e+00  1.883243e+02  1.928043e+02
 [6] -1.094219e+02 -6.638261e+02  1.740686e+01 -7.330081e-01  4.204461e+00
[11] -5.827103e+03  1.677997e+02  9.616624e-02  7.067210e+01 -4.855266e+03
[16] -3.479018e+00
[1] "Estimated Crime for new City"
[1] 155.4349
[1] "Estimated Crime for new City using new model"
         [,1]
[1,] 1112.678
